ACTOR 2 CRITIC with HYPERPARAMETER TUNING

Logan Wong

law3082

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

%cd /content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
!ls -la

/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
total 30
drwx------ 2 root root 4096 Sep 25 15:17 checkpoints
drwx------ 2 root root 4096 Sep 25 15:17 code
drwx------ 2 root root 4096 Sep 25 15:04 .git
-rw------- 1 root root 6305 Oct 16 19:53 github_terminal.ipynb
-rw------- 1 root root   33 Sep 26 19:25 .gitignore
drwx------ 2 root root 4096 Sep 25 15:17 models
-rw------- 1 root root 2348 Sep 29 02:21 README.md
drwx------ 2 root root 4096 Sep 25 15:17 results


In [3]:
!pip install gymnasium[atari,accept-rom-license] ale-py sb3_contrib stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 12.7 MB/s eta 0:00:00


In [4]:
import os
import torch
import gymnasium as gym
import stable_baselines3
import ale_py
import numpy as np

# RL Algorithm
from stable_baselines3 import A2C


# Visualization
from PIL import Image
import io
import base64
from IPython.display import display, HTML



# For debugging
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.callbacks import BaseCallback
import time

# Action masking
from gymnasium import ActionWrapper
from stable_baselines3.common.atari_wrappers import AtariWrapper

# Vector environment
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecEnvWrapper, DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

import gc
import random

print("All imports working")

All imports working


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
  print("CPU in use")

GPU available: True
GPU: Tesla T4


In [6]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [7]:
class ActionReducer(ActionWrapper):
  def __init__(self, env):
    super().__init__(env)

    # NOOP, FIRE, UP, and DOWN only. No UPFIRE. No DOWNFIRE.
    self.allowed_actions = [0,1,2,3]

    self.action_space = gym.spaces.Discrete(len(self.allowed_actions))

  def action(self, action):
    return self.allowed_actions[action]

In [8]:
def make_env():
  env = gym.make("ALE/Bowling-v5")
  env = ActionReducer(env)
  env = Monitor(env)
  # disable reward clipping
  env = AtariWrapper(env, clip_reward=False)
  return env

In [9]:
seed = 316
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

env = DummyVecEnv([make_env])
env = VecFrameStack(env, n_stack=4)
print("Env created")

Env created


In [10]:
def train_a2c_model(config_name, total_timesteps, learning_rate, gamma, n_steps, ent_coef):
    print(f"Testing {config_name}...")

    model = A2C(
        "CnnPolicy",
        env,

        learning_rate=learning_rate,
        gamma=gamma,

        n_steps=n_steps,
        ent_coef=ent_coef,

        gae_lambda=0.99,
        vf_coef=0.5,
        max_grad_norm=0.5,
        use_rms_prop=True,
        verbose=0,
        device="cuda"
    )

    # Train for a short time to compare
    model.learn(total_timesteps=total_timesteps, progress_bar=True)

    return model

In [11]:
def evaluate_model(model, env, n_eval_episodes):
    """Evaluate model and return mean reward"""
    rewards = []
    action_dict = {
      0: "NOOP",
      1: "FIRE",
      2: "UP",
      3: "DOWN"
    }
    for episode in range(n_eval_episodes):
        obs = env.reset()
        episode_reward = 0
        done = False
        total_reward = 0
        steps = 0

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            # Actions: NOOP(0), FIRE(1), UP(2), and DOWN(3)
            obs, reward, done, _ = env.step(action)
            # if reward[0] > 0:
            #   print(f"Reward earned for doing {action_dict[action[0]]}: {reward[0]}")
            episode_reward += reward[0]
            steps += 1

        rewards.append(episode_reward)
        print(f"Episode {episode+1}: Reward = {episode_reward:6.1f}, Steps = {steps}")

    return np.mean(rewards)

In [12]:
# Test different configurations

#     learning_rate=0.0001,
#     gamma=0.99,
#     n_steps=5,              # Steps per update; like DQN's train_freq=4
#     ent_coef=0.01,          # exploration (like epsilon-greedy)
#     gae_lambda=0.95,        # Standard value; provides slight bias for lower variance
#     vf_coef=0.5,            # Standard value; balance btwn actor & critic loss
# Vf_coef Balances actor (decision maker) and critic (value estimator). SO 0.5 balances both.


# gamma=0.999,          # Longer horizon for sparse rewards
# n_steps=10,           # More steps per update
# ent_coef=0.1,         # Much more exploration

# learning rate: 0.0001 = slow/steady vs. 0.001 = fast/jumpy
# n steps: Higher is better for sparse rewards. 5, 10, 15, 20, 25, 30, 35, 40, 45, 50
# ent coef: LOW (0.01) = less exploration, HIGH (0.1) = more exploration

# gae lambda:  Low (0.95) = Short-term matters more, High (0.99) = Long-term matters more (Better for Sparse)... I'll just keep it at 0.99
# vf coef: i'll just keep it at 0.5
# learning_rate, gamma, n_steps, ent_coef, gae_lambda
# configs = {
#     "conservative":   (0.0001, 5, 0.01, 0.95, 0.5),
#     "more_explore":   (0.0001, 5, 0.1, 0.95, 0.5),
#     "longer_horizon": (0.0001, 10, 0.01, 0.99, 0.5),
#     "higher_lr":      (0.001, 5, 0.01, 0.95, 0.5),
# }

# Learning rate: 0.0001 to 0.001
# gamma: 0.99 to 0.999
# n_steps: 5, 10, 15, 20, 25, 30, 35, 40, 45, 50
# ent_coef: 0.01 to 0.1
# game_lambda:0.95 to 0.99


configs = {}
config_id = 1

# Define the ranges and step sizes
learning_rates = [0.0001, 0.0005, 0.001]
gammas = [0.99, 0.995, 0.999]
n_steps_list = [5, 10, 15, 30, 40, 50]
ent_coefs = [0.01, 0.05, 0.1]

# Generate all combinations
for lr in learning_rates:
    for gamma in gammas:
        for n_steps in n_steps_list:
            for ent_coef in ent_coefs:
                config_name = f"config_{config_id}"
                configs[config_name] = (lr, gamma, n_steps, ent_coef)
                config_id += 1

print(f"Generated {len(configs)} configurations")
# for config in configs:
#   print(configs[config])


Generated 162 configurations


In [13]:
# total_timesteps = 10000000    # 10M
total_timesteps =   50000    # 50K
# total_timesteps =    10000    # 10K
# total_timesteps =     5000    # 5K

# n_eval_episodes = 10
n_eval_episodes = 30
# n_eval_episodes = 100

best_reward = -float('inf')
best_config = None

for name, params in configs.items():
    model = train_a2c_model(name, total_timesteps, *params)
    print("Model done training")

    reward = evaluate_model(model, env, n_eval_episodes)
    print(f"{name}: {reward:.4f} mean reward")

    if reward > best_reward:
        best_reward = reward
        best_config = name

  83% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━ 41,371/50,000  [ 0:04:10 < 0:00:54 , 162 it/s ]

In [ ]:
print(f"Best config: {best_config} with reward {best_reward}")

print(f"Learning rate: {best_config[0]}")
print(f"gamma: {best_config[1]}")
print(f"n steps: {best_config[2]}")
print(f"ent_coef: {best_config[3]}")

In [ ]:
torch.cuda.empty_cache()
del model
del env

In [ ]:
# TO DO:
# Run this to tune hyperparameters
# Assuming that trying out 162 configurations for 50K steps, and evaluating for 30 episodes
# Will find the best parameters
# That will translate to 10M steps and evaluating for 10K episodes
# So after I finish running this,
# Copy the best parameters found back into A2C.ipynb
# And run that for 10M steps
# THEN evaluate that saved model for 10K steps